In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
#import cv2
import tensorflow as tf
import tensorflow.keras.layers as L
from PIL import Image, ImageFile

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os
'''
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

'''

In [2]:
import tensorflow as tf

In [3]:
print(tf.version)

In [4]:
folder = '../input/cbir-dataset/dataset/'
photo = [folder+x for x in os.listdir(folder)]
photos = pd.DataFrame()
photos['dir'] = photo

In [5]:
def display(path_img):
    plt.imshow(plt.imread(path_img))
    plt.show()

In [6]:
display(photos['dir'][48])

In [7]:
img_arr=[]
for file in photos['dir']:
    img = Image.open(file)
    img = np.array(img.resize((224, 224)))
    img_arr.append(img)


In [8]:
import pickle
# store features in pickle
pickle.dump(img_arr, open(os.path.join('./', 'images.pkl'), 'wb'))

In [9]:
img_arr = np.array(img_arr)/255.0
img_arr.shape

In [10]:
img_arr.max()

In [11]:
plt.imshow(img_arr[1])
plt.show()

In [12]:
model = tf.keras.Sequential([
    L.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)),
    L.Conv2D(64, (3, 3), padding='same', activation='relu'),
    L.MaxPooling2D((2, 2), strides=2, padding='same'),
    L.Conv2D(128, (3, 3), padding='same', activation='relu'),
    L.Conv2D(128, (3, 3), padding='same', activation='relu'),
    L.MaxPooling2D((2, 2), strides=2, padding='same'),
    L.Conv2D(256, (3, 3), padding='same', activation='relu'),
    L.Conv2D(256, (3, 3), padding='same', activation='relu'),
    L.Conv2D(256, (3, 3), padding='same', activation='relu'),
    L.MaxPooling2D((2, 2), strides=2, padding='same'),
    L.Conv2D(512, (3, 3), padding='same', activation='relu'),
    L.Conv2D(512, (3, 3), padding='same', activation='relu'),
    L.Conv2D(512, (3, 3), padding='same', activation='relu'),
    L.MaxPooling2D((2, 2), strides=2, padding='same'),
    L.Conv2D(512, (3, 3), padding='same', activation='relu'),
    L.Conv2D(512, (3, 3), padding='same', activation='relu'),
    L.Conv2D(512, (3, 3), padding='same', activation='relu'),
    L.MaxPooling2D((2, 2), strides=2, padding='same'),
    L.Flatten(),
    L.Dense(4096, activation='relu'),
    L.Dense(4096, activation='relu')
])

In [13]:
model.summary()

In [14]:
#image = preprocess_input(img_arr)
    # extract features
feature = model.predict(img_arr, verbose=1)

In [15]:
feature[0]

In [16]:
model.save('model.h5')

In [17]:
import pickle
# store features in pickle
pickle.dump(feature, open(os.path.join('./', 'features.pkl'), 'wb'))

In [18]:
def euclidean(a, b):
    # compute and return the euclidean distance between two vectors
    return np.linalg.norm(a - b)

In [19]:
def perform_search(queryFeatures, index, maxResults=64):

    results = []

    for i in range(0, len(index)):
        # compute the euclidean distance between our query features
        # and the features for the current image in our index, then
        # update our results list with a 2-tuple consisting of the
        # computed distance and the index of the image
        d = euclidean(queryFeatures, index[i])
        results.append((d, i))
    
    # sort the results and grab the top ones
    results = sorted(results)[:maxResults]
    # return the list of results
    return results

In [27]:
index = 9
query = img_arr[index].reshape((1,224,224,3))
query = model.predict(query)

In [28]:
result = perform_search(query, feature, 20)
arr = []
for a, b in result:
    arr.append(b)
fig = plt.figure(figsize=(16, 16))
for i in range(1,20):
    #img = plt.imread(photo[arr[i-1]])
    fig.add_subplot(5, 5, i)
    plt.imshow(img_arr[arr[i-1]])
plt.show()